# RE計算

## Rebas

In [1]:
import pandas as pd
import numpy as np

In [2]:
excel_file_path1 = 'C:/Users/ASUS/OneDrive/Documents/實習/Rebas/CPBL_2018.xlsx'
excel_file_path2 = 'C:/Users/ASUS/OneDrive/Documents/實習/Rebas/CPBL_2019.xlsx'# 請替換成你的 Excel 檔案路徑
excel_file_path3 = 'C:/Users/ASUS/OneDrive/Documents/實習/Rebas/CPBL_2020.xlsx'
excel_file_path4 = 'C:/Users/ASUS/OneDrive/Documents/實習/Rebas/CPBL_2021.xlsx'
excel_file_path5 = 'C:/Users/ASUS/OneDrive/Documents/實習/Rebas/CPBL_2022.xlsx'
excel_file_path6 = 'C:/Users/ASUS/OneDrive/Documents/實習/Rebas/CPBL_2023.xlsx'
sheet_name1 = 'AWAY-打席資訊'  # 請替換成你要載入的工作表名稱
sheet_name2 = 'HOME-打席資訊'

In [3]:
away_2018 = pd.read_excel(excel_file_path1, sheet_name=sheet_name1)
home_2018 = pd.read_excel(excel_file_path1, sheet_name=sheet_name2)

away_2019 = pd.read_excel(excel_file_path2, sheet_name=sheet_name1)
home_2019 = pd.read_excel(excel_file_path2, sheet_name=sheet_name2)

away_2020 = pd.read_excel(excel_file_path3, sheet_name=sheet_name1)
home_2020 = pd.read_excel(excel_file_path3, sheet_name=sheet_name2)

away_2021 = pd.read_excel(excel_file_path4, sheet_name=sheet_name1)
home_2021 = pd.read_excel(excel_file_path4, sheet_name=sheet_name2)

away_2022 = pd.read_excel(excel_file_path5, sheet_name=sheet_name1)
home_2022 = pd.read_excel(excel_file_path5, sheet_name=sheet_name2)

away_2023 = pd.read_excel(excel_file_path6, sheet_name=sheet_name1)
home_2023 = pd.read_excel(excel_file_path6, sheet_name=sheet_name2)


In [4]:
data1 = pd.concat([away_2018, away_2019], ignore_index=True)
data2 = pd.concat([data1, away_2020], ignore_index=True)
data3 = pd.concat([data2, away_2021], ignore_index=True)
data4 = pd.concat([data3, away_2022], ignore_index=True)
away_stat = pd.concat([data4, away_2023], ignore_index=True)

In [5]:
away_stat['得分']=away_stat['結束客場分']-away_stat['開始客場分']

In [6]:
#主場
data1 = pd.concat([home_2018, home_2019], ignore_index=True)
data2 = pd.concat([data1, home_2020], ignore_index=True)
data3 = pd.concat([data2, home_2021], ignore_index=True)
data4 = pd.concat([data3, home_2022], ignore_index=True)
home_stat = pd.concat([data4, home_2023], ignore_index=True)

In [7]:
home_stat['得分']=home_stat['結束主場分']-home_stat['開始主場分']

In [8]:
col = ['比賽編號','比賽日期','局數','得分','出局數','壘包狀態','好球','壞球']

In [9]:
home_stat=home_stat[col]
away_stat=away_stat[col]

In [10]:
data = pd.concat([away_stat, home_stat], ignore_index=True)
#data

In [11]:
# 計算該比賽該局的最終得分
data['總得分'] = data.groupby(['比賽編號','比賽日期','局數'])[['得分']].transform('max')

# 計算當下狀態的得分變化
#df['得分變化'] = df.groupby(['gameId', 'inning'])[['awayScores', 'homeScores']].diff().fillna(0)

# 聚合以計算每種狀態的期望得分
run_expectancy1 = data.groupby(['壘包狀態', '壞球', '好球', '出局數']).agg(
    出現次數=('得分', 'count'),
    總得分=('總得分', 'sum')
).reset_index()

# 計算得分期望值
#run_expectancy1['得分期望'] = run_expectancy1['總得分'] / run_expectancy1['出現次數']
run_expectancy1

,壘包狀態,壞球,好球,出局數,出現次數,總得分
0,0,0,0,0,2674,1867
1,0,0,0,1,1830,1148
2,0,0,0,2,1472,961
3,0,0,0,3,2,0
4,0,0,1,0,2851,1940
...,...,...,...,...,...,...
290,7,3,1,1,62,97
291,7,3,1,2,74,120
292,7,3,2,0,65,109
293,7,3,2,1,139,206


## 紀錄系統

In [12]:
excel_file_path1 = 'C:/Users/ASUS/OneDrive/Documents/中信兄弟/Season_data/2024_non_trackman.xlsx'
sheet_name = 'Sheet1'

In [13]:
data_2024 = pd.read_excel(excel_file_path1, sheet_name=sheet_name)

In [14]:
non_track = data_2024

In [15]:
def base_state(row):
    state = 0
    if not pd.isna(row['firstId']):
        state += 1
    if not pd.isna(row['secondId']):
        state += 2
    if not pd.isna(row['thirdId']):
        state += 4
    return state

non_track['壘包狀態'] = non_track.apply(base_state, axis=1)

In [16]:
# 計算該比賽該局的最終得分
non_track['總得分'] = non_track.groupby(['gameId', 'inning'])[['runsOnPlay']].transform('max')

# 計算當下狀態的得分變化
#df['得分變化'] = df.groupby(['gameId', 'inning'])[['awayScores', 'homeScores']].diff().fillna(0)

# 聚合以計算每種狀態的期望得分
run_expectancy2 = non_track.groupby(['壘包狀態', 'balls', 'strikes', 'outs']).agg(
    出現次數=('runsOnPlay', 'count'),
    總得分=('總得分', 'sum')
).reset_index()

# 計算得分期望值
#run_expectancy2['得分期望'] = run_expectancy2['總得分'] / run_expectancy2['出現次數']
run_expectancy2= run_expectancy2.rename(columns={'outs': '出局數','balls': '壞球','strikes': '好球'})
run_expectancy2

,壘包狀態,壞球,好球,出局數,出現次數,總得分
0,0,0,0,0,6393,3934
1,0,0,0,1,4329,2307
2,0,0,0,2,3265,1626
3,0,0,1,0,3323,1976
4,0,0,1,1,2230,1164
...,...,...,...,...,...,...
286,7,3,1,1,24,30
287,7,3,1,2,25,36
288,7,3,2,0,14,16
289,7,3,2,1,55,69


In [18]:
# 合併兩個表格並加總
merge_df = pd.concat([run_expectancy1, run_expectancy2]).groupby(['壘包狀態', '壞球', '好球', '出局數'], as_index=False).sum()
merge_df = merge_df.query("出局數 != 3")
merge_df['Expected_Runs'] = merge_df['總得分'] / merge_df['出現次數']
merge_df['比例'] = merge_df['出現次數'] / merge_df['出現次數'].sum()
merge_df['加權得分期望值'] = merge_df['Expected_Runs'] * merge_df['比例']
merge_df

,壘包狀態,壞球,好球,出局數,出現次數,總得分,Expected_Runs,比例,加權得分期望值
0,0,0,0,0,9067,5801,0.639793,0.039081,0.025003
1,0,0,0,1,6159,3455,0.560968,0.026546,0.014892
2,0,0,0,2,4737,2587,0.546126,0.020417,0.011150
4,0,0,1,0,6174,3916,0.634273,0.026611,0.016879
5,0,0,1,1,4105,2365,0.576127,0.017693,0.010194
...,...,...,...,...,...,...,...,...,...
293,7,3,1,1,86,127,1.476744,0.000371,0.000547
294,7,3,1,2,99,156,1.575758,0.000427,0.000672
295,7,3,2,0,79,125,1.582278,0.000341,0.000539
296,7,3,2,1,194,275,1.417526,0.000836,0.001185


In [53]:
#merge_df.to_excel('RE288.xlsx', index=True)

## Pittch Value 計算

In [19]:
name_mapping = {
    'SL':'Slider',
    'SI':'Sinker', 
    'FF':'Fastball', 
    'CH':'ChangeUp', 
    'CU':'Curveball', 
    'FO':'Splitter', 
    'FC':'Cutter',
    'KN':'Knuckle curve',
    'FS':'Splitter'
    }

# 新增一列"球員"，根据"Pitcher"列的值匹配中文名
data_2024 ['球種'] = data_2024 ['pType'].map(name_mapping)

In [20]:
data_2024 ['PitchofPA'] = data_2024 .groupby(['bName', 'pName', 'date', 'gameId', 'paRound', 'paOrder']).cumcount() + 1

In [61]:
data_2024.sort_values(by=['date','inning','gameId', 'paRound', 'paOrder','pName', 'bName', 'PitchofPA'], inplace=True)
def get_expected_runs(row):
    base_type = row["壘包狀態"]
    outs = row["outs"]
    balls = row["balls"]
    strikes = row["strikes"]
    
    matching_row = merge_df[(merge_df["壘包狀態"] == base_type) & 
                          (merge_df["出局數"] == outs) & 
                          (merge_df["壞球"] == balls) & 
                          (merge_df["好球"] == strikes)]
    
    if not matching_row.empty:
        return matching_row.iloc[0]["Expected_Runs"]
    else:
        return None

data_2024 ["Expected_Runs"] = data_2024.apply(get_expected_runs, axis=1)
data_2024["Next_Expected_Runs"] = data_2024.groupby(["date", "gameId",'inning','side'])["Expected_Runs"].shift(-1)
data_2024["Next_Expected_Runs"].fillna(0, inplace=True)

data_2024 

C:\Users\ASUS\AppData\Local\Temp\ipykernel_16332\2771945053.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_2024["Next_Expected_Runs"].fillna(0, inplace=True)


,LocalDateTime,id,gameId,stadium,umpireInChief,date,eType,awayScores,homeScores,inning,...,fielder_DH,壘包狀態,總得分,球種,PitchofPA,Expected_Runs,RV,Next_Expected_Runs,RV2,人數
13,'2024-03-30T17:15:43.9430+08:00,66081720133ed988f9c51ed8,6607d24cdf5ba4d17362f7ee,臺北大巨蛋 Taipei Dome,張展榮,2024-03-30T08:49:00.000Z,P,0,0,1,...,NaN,0,0,Fastball,1,0.639793,0.005520,0.634273,0.005520,0
14,'2024-03-30T17:15:59.0230+08:00,66081730133ed988f9c51ed9,6607d24cdf5ba4d17362f7ee,臺北大巨蛋 Taipei Dome,張展榮,2024-03-30T08:49:00.000Z,P,0,0,1,...,NaN,0,0,Fastball,2,0.634273,0.006434,0.627839,0.006434,0
15,'2024-03-30T17:16:16.4930+08:00,66081763133ed988f9c51eda,6607d24cdf5ba4d17362f7ee,臺北大巨蛋 Taipei Dome,張展榮,2024-03-30T08:49:00.000Z,P,0,0,1,...,NaN,0,0,Fastball,3,0.627839,-0.011954,0.560968,-0.011954,0
0,'2024-03-30T17:07:23.6810+08:00,66081466133ed988f9c51ecb,6607d24cdf5ba4d17362f7ee,臺北大巨蛋 Taipei Dome,張展榮,2024-03-30T08:49:00.000Z,P,0,0,1,...,NaN,0,0,Fastball,1,0.639793,-0.035282,0.675075,-0.035282,0
1,'2024-03-30T17:07:49.7300+08:00,660814a9133ed988f9c51ecc,6607d24cdf5ba4d17362f7ee,臺北大巨蛋 Taipei Dome,張展榮,2024-03-30T08:49:00.000Z,P,0,0,1,...,NaN,0,0,Fastball,2,0.675075,0.114107,0.560968,0.114107,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37451,'2024-10-11T21:27:52.2720+08:00,670e0ac31a3ad7da85c245df,670e0ac31a3ad7da85c245da,樂天桃園棒球場 Rakuten Taoyuan Baseball Stadium,羅鈞鴻,2024-10-11T10:35:00.000Z,P,0,6,9,...,NaN,0,0,Fastball,1,0.546126,-0.023762,0.569888,-0.023762,0
37452,'2024-10-11T21:28:07.4150+08:00,670e0ac31a3ad7da85c245de,670e0ac31a3ad7da85c245da,樂天桃園棒球場 Rakuten Taoyuan Baseball Stadium,羅鈞鴻,2024-10-11T10:35:00.000Z,P,0,6,9,...,NaN,0,0,Fastball,2,0.569888,0.014004,0.555885,0.014004,0
37453,'2024-10-11T21:28:27.5690+08:00,670e0ac31a3ad7da85c245dd,670e0ac31a3ad7da85c245da,樂天桃園棒球場 Rakuten Taoyuan Baseball Stadium,羅鈞鴻,2024-10-11T10:35:00.000Z,P,0,6,9,...,NaN,0,0,Fastball,3,0.555885,0.030651,0.525234,0.030651,0
37454,'2024-10-11T21:28:43.9600+08:00,670e0ac31a3ad7da85c245dc,670e0ac31a3ad7da85c245da,樂天桃園棒球場 Rakuten Taoyuan Baseball Stadium,羅鈞鴻,2024-10-11T10:35:00.000Z,P,0,6,9,...,NaN,0,0,Slider,4,0.525234,-0.026856,0.552089,-0.026856,0


In [65]:
data_2024['RV'] = data_2024["Expected_Runs"] - data_2024["Next_Expected_Runs"] - data_2024["runsOnPlay"]

In [66]:
#data_2024.sort_values(by=['date','inning','gameId', 'paRound', 'paOrder','pName', 'bName', 'PitchofPA'], inplace=True)

# 计算每个球之间预期分数的变化
#data_2024['RV'] = -data_2024['Expected_Runs'].diff()

# 将PitchofPA为1的RV定为0
#data_2024.loc[data_2024['PitchofPA'] == 1, 'RV'] = 0

# 显示处理后的数据
data_2024

,LocalDateTime,id,gameId,stadium,umpireInChief,date,eType,awayScores,homeScores,inning,...,fielder_DH,壘包狀態,總得分,球種,PitchofPA,Expected_Runs,RV,Next_Expected_Runs,RV2,人數
13,'2024-03-30T17:15:43.9430+08:00,66081720133ed988f9c51ed8,6607d24cdf5ba4d17362f7ee,臺北大巨蛋 Taipei Dome,張展榮,2024-03-30T08:49:00.000Z,P,0,0,1,...,NaN,0,0,Fastball,1,0.639793,0.005520,0.634273,0.005520,0
14,'2024-03-30T17:15:59.0230+08:00,66081730133ed988f9c51ed9,6607d24cdf5ba4d17362f7ee,臺北大巨蛋 Taipei Dome,張展榮,2024-03-30T08:49:00.000Z,P,0,0,1,...,NaN,0,0,Fastball,2,0.634273,0.006434,0.627839,0.006434,0
15,'2024-03-30T17:16:16.4930+08:00,66081763133ed988f9c51eda,6607d24cdf5ba4d17362f7ee,臺北大巨蛋 Taipei Dome,張展榮,2024-03-30T08:49:00.000Z,P,0,0,1,...,NaN,0,0,Fastball,3,0.627839,0.066871,0.560968,-0.011954,0
0,'2024-03-30T17:07:23.6810+08:00,66081466133ed988f9c51ecb,6607d24cdf5ba4d17362f7ee,臺北大巨蛋 Taipei Dome,張展榮,2024-03-30T08:49:00.000Z,P,0,0,1,...,NaN,0,0,Fastball,1,0.639793,-0.035282,0.675075,-0.035282,0
1,'2024-03-30T17:07:49.7300+08:00,660814a9133ed988f9c51ecc,6607d24cdf5ba4d17362f7ee,臺北大巨蛋 Taipei Dome,張展榮,2024-03-30T08:49:00.000Z,P,0,0,1,...,NaN,0,0,Fastball,2,0.675075,0.114107,0.560968,0.114107,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37451,'2024-10-11T21:27:52.2720+08:00,670e0ac31a3ad7da85c245df,670e0ac31a3ad7da85c245da,樂天桃園棒球場 Rakuten Taoyuan Baseball Stadium,羅鈞鴻,2024-10-11T10:35:00.000Z,P,0,6,9,...,NaN,0,0,Fastball,1,0.546126,-0.023762,0.569888,-0.023762,0
37452,'2024-10-11T21:28:07.4150+08:00,670e0ac31a3ad7da85c245de,670e0ac31a3ad7da85c245da,樂天桃園棒球場 Rakuten Taoyuan Baseball Stadium,羅鈞鴻,2024-10-11T10:35:00.000Z,P,0,6,9,...,NaN,0,0,Fastball,2,0.569888,0.014004,0.555885,0.014004,0
37453,'2024-10-11T21:28:27.5690+08:00,670e0ac31a3ad7da85c245dd,670e0ac31a3ad7da85c245da,樂天桃園棒球場 Rakuten Taoyuan Baseball Stadium,羅鈞鴻,2024-10-11T10:35:00.000Z,P,0,6,9,...,NaN,0,0,Fastball,3,0.555885,0.030651,0.525234,0.030651,0
37454,'2024-10-11T21:28:43.9600+08:00,670e0ac31a3ad7da85c245dc,670e0ac31a3ad7da85c245da,樂天桃園棒球場 Rakuten Taoyuan Baseball Stadium,羅鈞鴻,2024-10-11T10:35:00.000Z,P,0,6,9,...,NaN,0,0,Slider,4,0.525234,-0.026856,0.552089,-0.026856,0


In [67]:
rv_sum = data_2024.groupby(['pName', '球種'])['RV'].sum()

pitch_count = data_2024.groupby(['pName', '球種']).size()

result_df = pd.DataFrame({'PV_sum': rv_sum, 'PitchCount': pitch_count})

result_df['PV/100']=result_df['PV_sum']/result_df['PitchCount']*100

result_df.reset_index(inplace=True)

result_df

,pName,球種,PV_sum,PitchCount,PV/100
0,伍祐城,ChangeUp,-3.394147,43,-7.893364
1,伍祐城,Curveball,-1.183995,52,-2.276913
2,伍祐城,Cutter,-0.457152,39,-1.172184
3,伍祐城,Fastball,12.767989,421,3.032776
4,伍祐城,Slider,-0.049744,189,-0.026320
...,...,...,...,...,...
757,黃群,Curveball,2.469668,111,2.224927
758,黃群,Cutter,1.990467,25,7.961868
759,黃群,Fastball,-1.795675,398,-0.451175
760,黃群,Slider,3.177733,111,2.862822


In [68]:
#result_df.to_excel('2024_PV估算2.xlsx', index=True)

In [57]:
rv_mean = data_2024.groupby(['球種']).agg(PV_sum=('RV', 'sum'), PitchCount=('RV', 'count')).reset_index()
rv_mean['PV/100']=rv_mean['PV_sum']/rv_mean['PitchCount']*100
rv_mean

,球種,PV_sum,PitchCount,PV/100
0,ChangeUp,-4.665015,11045,-0.042236
1,Curveball,-25.367871,9653,-0.262798
2,Cutter,2.568033,5523,0.046497
3,Fastball,142.100090,49749,0.285634
4,Knuckle curve,-0.309883,22,-1.408558
5,Sinker,22.288311,3744,0.595307
6,Slider,82.493981,16914,0.487726
7,Splitter,11.286515,6175,0.182778


In [ ]:
#result_df['PV/C +']=result_df['PV/100']/result_df['PitchCount']*100